In [1]:
import numpy as np
import optuna
from sklearn import metrics
import warnings
import pickle
warnings.simplefilter(action='ignore', category=FutureWarning)
import joblib
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, r2_score, roc_curve
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from torch.optim import lr_scheduler

from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch.utils.data.dataloader import default_collate

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

c:\Users\14390\.conda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the data
data = np.load('DataSet/Classified_Data_zone.npz')

Features = data['features']
Labels = data['labels']

print(Features.shape, Labels.shape)

(16871, 9, 3, 3) (16871,)


In [3]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=2, stride=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2, stride=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=1)  # Max pooling to go from 2x2 to 1x1
        x = F.relu(self.conv2(x))
        return x

In [122]:
class CustomAttention(nn.Module):
    def __init__(self, feature_dim, time_steps):
        super(CustomAttention, self).__init__()
        self.query = nn.Linear(feature_dim, feature_dim)
        self.key = nn.Linear(feature_dim, feature_dim)
        self.value = nn.Linear(feature_dim, feature_dim)
        self.time_steps = time_steps

    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)

        # 初始化attention_scores为负无穷大，确保未更新的元素在softmax后贡献为0
        attention_scores = torch.full((x.size(0), self.time_steps, self.time_steps), float('-inf'), device=x.device)
        
        # 更新注意力分数，考虑每个9的倍数起始点
        for start in range(9):  # 从0到8，为每个可能的序列起始点
            for i in range(start, self.time_steps, 9):
                for j in range(start, self.time_steps, 9):
                    if i != j:  # 确保不是自己到自己
                        attention_scores[:, i, j] = torch.sum(queries[:, i] * keys[:, j], dim=-1)

        attention = F.softmax(attention_scores, dim=-1)
        attended_values = torch.bmm(attention, values)
        return attended_values

In [123]:
class ClassifyCNN(nn.Module):
    def __init__(self):
        super(ClassifyCNN, self).__init__()
        self.conv_block = ConvBlock()
        self.attention = CustomAttention(feature_dim=32, time_steps=27)
        self.fc = nn.Linear(32, 2)  # 保留为2，因为我们使用交叉熵损失函数

    def forward(self, x, return_logits=False):
        batch_size, time_steps, _, _ = x.shape
        x = x.view(batch_size * time_steps, 1, 3, 3)
        x = self.conv_block(x)
        x = x.view(batch_size, time_steps, -1)
        x = self.attention(x)
        x = torch.mean(x, dim=1)
        logits = self.fc(x)
        if return_logits:
            return logits  # 在训练时返回原始 logits
        return torch.argmax(logits, dim=1, keepdim=True)  # 在推理时返回类别索引

In [9]:
class ClassifyCNN(nn.Module):
    def __init__(self):
        super(ClassifyCNN, self).__init__()
        self.conv1 = nn.Conv2d(9, 16, kernel_size=2, stride=1)
        self.pool = nn.MaxPool2d(2, stride=1)
        self.fc1 = nn.Linear(16 * 1 * 1, 32)  # 正确调整全连接层的输入尺寸
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x, return_logits=False):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 1 * 1)  # 正确调整view的参数
        x = F.relu(self.fc1(x))
        logits = torch.sigmoid(self.fc2(x))
        if return_logits:
            return logits  # 在训练时返回原始 logits
        return torch.argmax(logits, dim=1, keepdim=True)  # 在推理时返回类别索引

In [14]:
model = ClassifyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 每隔30个epoch，学习率乘以0.1                                                                                                      
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [15]:
# 分割数据集，通常保留20%作为测试集
train_features, test_features, train_labels, test_labels = train_test_split(
    Features, Labels, test_size=0.2, random_state=42)

# 转换为 torch.tensor
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)

# 创建训练 DataLoader
train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
batch_size = 64  # 设置batch_size为1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 对测试数据进行相同的处理
test_features_tensor = torch.tensor(test_features, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# 创建测试 DataLoader
test_dataset = TensorDataset(test_features_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
num_epochs = 100  # 可以根据模型表现和计算资源进行调整

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()

# 训练函数
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device).long()
        optimizer.zero_grad()
        output = model(data, return_logits=True)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * data.size(0)
    
    scheduler.step()  # 更新学习率

    model.eval()
    total_test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device).long()
            output = model(data, return_logits=True)
            loss = criterion(output, target)
            total_test_loss += loss.item() * data.size(0)

    avg_train_loss = total_train_loss / len(train_loader.dataset)
    avg_test_loss = total_test_loss / len(test_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')
    print(f'Current Learning Rate: {scheduler.get_last_lr()[0]}')

Epoch 1/100, Train Loss: 0.5128, Test Loss: 0.4911
Current Learning Rate: 0.01
Epoch 2/100, Train Loss: 0.4903, Test Loss: 0.4884
Current Learning Rate: 0.01
Epoch 3/100, Train Loss: 0.4895, Test Loss: 0.4856
Current Learning Rate: 0.01
Epoch 4/100, Train Loss: 0.4868, Test Loss: 0.4881
Current Learning Rate: 0.01
Epoch 5/100, Train Loss: 0.4852, Test Loss: 0.4853
Current Learning Rate: 0.01
Epoch 6/100, Train Loss: 0.4851, Test Loss: 0.4861
Current Learning Rate: 0.01
Epoch 7/100, Train Loss: 0.4839, Test Loss: 0.4853
Current Learning Rate: 0.01
Epoch 8/100, Train Loss: 0.4832, Test Loss: 0.4859
Current Learning Rate: 0.01
Epoch 9/100, Train Loss: 0.4826, Test Loss: 0.4869
Current Learning Rate: 0.01
Epoch 10/100, Train Loss: 0.4825, Test Loss: 0.4857
Current Learning Rate: 0.01
Epoch 11/100, Train Loss: 0.4823, Test Loss: 0.4856
Current Learning Rate: 0.01
Epoch 12/100, Train Loss: 0.4811, Test Loss: 0.4889
Current Learning Rate: 0.01
Epoch 13/100, Train Loss: 0.4808, Test Loss: 0.48

In [18]:
model.eval()
y_pred = []
y_test = []


with torch.no_grad():
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)

        # 进行预测
        output = model(data)
        preds = output
        y_pred.extend(preds.cpu().numpy())
        y_test.extend(target.cpu().numpy())

y_pred = np.array(y_pred).flatten()
y_test = np.array(y_test).flatten()

In [19]:
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

              precision    recall  f1-score   support

         0.0       0.86      0.79      0.82      1872
         1.0       0.76      0.84      0.80      1503

    accuracy                           0.81      3375
   macro avg       0.81      0.81      0.81      3375
weighted avg       0.82      0.81      0.81      3375

Accuracy: 0.8118518518518518


In [76]:
def custom_collate(batch):
    batch = default_collate(batch)  # 首先使用默认的方式合并数据
    batch = (item.squeeze(0) for item in batch)  # 去除每个item的批次维度
    return batch

# 分割数据集，通常保留20%作为测试集
train_features, test_features, train_labels, test_labels = train_test_split(
    Features, Labels, test_size=0.2, random_state=42)

# 转换为 torch.tensor
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)

# 对测试数据进行相同的处理
test_features_tensor = torch.tensor(test_features, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# 使用自定义的 collate_fn 创建 DataLoader
train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_features_tensor, test_labels_tensor)
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=custom_collate)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [72]:
class GATClassifier(torch.nn.Module):
    def __init__(self):
        super(GATClassifier, self).__init__()
        self.edge_index = self.build_adjacency_matrix().long()
        self.conv1 = GATConv(in_channels=9, out_channels=16, heads=4, dropout=0.6)
        self.conv2 = GATConv(in_channels=64, out_channels=16, heads=1, dropout=0.6)
        self.fc = torch.nn.Linear(16 * 9, 2)  # 假设是9个节点

    def build_adjacency_matrix(self):
        # 建立邻接矩阵的逻辑
        indices = []
        size = 3
        for i in range(size):
            for j in range(size):
                index = i * size + j
                if j < size - 1:
                    indices.append((index, index + 1))  # 右
                if i < size - 1:
                    indices.append((index, index + size))  # 下
                if i < size - 1 and j < size - 1:
                    indices.append((index, index + size + 1))  # 右下角
                if i < size - 1 and j > 0:
                    indices.append((index, index + size - 1))  # 左下角
        return torch.tensor(indices).t().contiguous()

    def forward(self, x):
        # 假设 x 的初始形状是 (9, 3, 3)，我们需要将其转换为 (9, 9)
        x = x.view(9, -1)  # 将每个3x3的矩阵拉平成一个长度为9的向量
        x = F.relu(self.conv1(x, self.edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.conv2(x, self.edge_index))
        x = x.view(-1)
        x = self.fc(x)
        return x

In [73]:
# 初始化模型和优化器
model = GATClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [74]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        target = target.long()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            target = target.long()
            outputs = model(data)
            _, predicted = torch.max(outputs, dim=0)
            correct += (predicted == target).sum().item()
            total += 1
    return correct / total

In [75]:
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_acc = test(model, test_loader)
    print(f'Epoch: {epoch}/{num_epochs}, Training Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

Epoch: 1/50, Training Loss: 0.8648, Test Accuracy: 0.7556
Epoch: 2/50, Training Loss: 1.0231, Test Accuracy: 0.5547
Epoch: 3/50, Training Loss: 0.8310, Test Accuracy: 0.5547
Epoch: 4/50, Training Loss: 0.7845, Test Accuracy: 0.5547
Epoch: 5/50, Training Loss: 0.7538, Test Accuracy: 0.5547


KeyboardInterrupt: 